**Imports** - *External*

In [1]:
%matplotlib widget
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import scipy
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import common

*Internal*

In [2]:
from Data import Moca # This file alrzeady has the data converted from string to numeric

MMSE ready!
   Phase  ID  RID  SITEID VISCODE VISCODE2  MMDATE  MMYEAR  MMMONTH  MMDAY  \
0      1  10    2     107      sc       sc       1       1        1      1   
1      1  12    1      10       f        f       1       1        1      1   
2      1  14    3     107      sc       sc       0       1        0      0   
3      1  16    4      10      sc       sc       1       1        1      1   
4      1  18    5     107      sc       sc       1       1        1      1   

   ...  MMWATCH  MMPENCIL  MMREPEAT  MMHAND  MMFOLD  MMONFLR  MMREAD  MMWRITE  \
0  ...        1         1         1       1       1        1       1        1   
1  ...        1         1         1       1       1        1       1        1   
2  ...        1         1         1       1       1        1       1        1   
3  ...        1         1         1       1       1        1       1        1   
4  ...        1         1         1       0       1        1       1        1   

   MMDRAW  MMSCORE  
0       1  

**There are some errors while calculating the correlation, these are shown as warnigns. The next line will ignore these errors**

In [3]:
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

**Fnction used to calculate correlation between two columns where they both have some sort of value:**

This function is using Pearson product-moment correlation coefficients:

\begin{equation}
    R_{ij} = \frac{ C_{ij} } { \sqrt{ C_{ii} * C_{jj} } }
\end{equation}


In [4]:
def getCorrP(x, y):
    xData = []
    yData = []
    for i in range(len(x)): # Both columns must have same size
        if x[i] != '' and y[i] != '':
            xData.append(float(x[i]))
            yData.append(float(y[i]))
    if len(xData) > 1:
        return pearsonr(xData, yData)
    return 0, 0
    

In [5]:
data = Moca().data

unusedLables = ['Phase', 'ID', 'RID', 'SITEID', 'VISCODE2']
data = data.drop(columns=unusedLables)

dataLabels = data.columns.values
dataNumpy = data.to_numpy()

correlations = pd.DataFrame(index=dataLabels, columns=dataLabels)

pValues = pd.DataFrame(index=dataLabels, columns=dataLabels)

tableCorrP = pd.DataFrame(columns=["Labels", "Correlation", "p_Value"])




for i in range(len(dataLabels)):
    correlations.at[dataLabels[i], dataLabels[i]] = 1
    pValues.at[dataLabels[i], dataLabels[i]] = 0
    for j in range(i+1, len(dataLabels)):
        p_r, p_p = getCorrP(dataNumpy[:, i], dataNumpy[:, j])
        correlations.at[dataLabels[i], dataLabels[j]] = correlations.at[dataLabels[j], dataLabels[i]] = p_r
        pValues.at[dataLabels[i], dataLabels[j]] = pValues.at[dataLabels[j], dataLabels[i]] = p_p
        
        tableCorrP = tableCorrP.append(dict(zip(tableCorrP.columns,[f"{dataLabels[i]} -> {dataLabels[j]}", p_r, p_p])), ignore_index = True)
        
        


 
    
    



E:\Projects\CognitiveProfiling_ADNI\Data\tableScripts\Moca.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[field] = [int(x) for x in data[field]]


Moca ready!


In [6]:
print(tableCorrP.head(10))

               Labels  Correlation       p_Value
0      TRAILS -> CUBE     0.218048  1.571643e-19
1  TRAILS -> CLOCKCON     0.198721  2.015009e-16
2   TRAILS -> CLOCKNO     0.294963  4.474477e-35
3  TRAILS -> CLOCKHAN     0.245159  2.044025e-24
4      TRAILS -> LION     0.026057  2.857947e-01
5     TRAILS -> RHINO     0.090901  1.908972e-04
6     TRAILS -> CAMEL     0.117891  1.263635e-06
7   TRAILS -> IMMT1W1     0.101856  2.881615e-05
8   TRAILS -> IMMT1W2     0.155398  1.517363e-10
9   TRAILS -> IMMT1W3     0.138921  1.080678e-08


In [7]:
aux = sns.color_palette("coolwarm", as_cmap=True) #Changes the color of the graph

# Correlations

In [8]:
plt.close()
sns.heatmap(np.ma.filled(correlations.astype(float), np.nan), annot=True, annot_kws={"size": 7}, xticklabels=correlations.columns, yticklabels=correlations.index, cmap=aux)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

# P_Values

In [10]:
plt.close()
sns.heatmap(np.ma.filled(pValues.astype(float), np.nan), annot=True, annot_kws={"size": 7}, xticklabels=pValues.columns, yticklabels=pValues.index, cmap=aux)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>